In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
from pypdf import PdfReader
import requests
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def push(message):
    print(f"Pushing : {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(url=pushover_url, data=payload)


In [5]:
push("heyy!!!")

Pushing : heyy!!!


In [6]:
def record_user_details(email, name="Name not provided", note="Note not provided"):
    push(f"Recording interest from : {name}, with email: {email} and note: {note}")
    return { "recorded": "ok" }

In [7]:
def record_unknown_question(question):
    push(f"Recording this question: {question} which i could not provide any answer")
    return { "recorded": "ok" }

In [8]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "note": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [9]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Use this tool to record that a user asked a question that could not be answered as you did not know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [10]:
tools = [{"type": "function", "function": record_user_details_json }, 
            {"type": "function", "function": record_unknown_question_json}]
    

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'note': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Use this tool to record that a user asked a question that could not be answered as you did not know the answer',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required

In [12]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name} with this arguments: {arguments}", flush=True)

        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})

    return results


In [13]:
reader = PdfReader("../Profile.pdf")

linkedin = "" 

for page in reader.pages:
    text = page.extract_text()
    linkedin += text

with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Soroush Alahvaisi"

In [22]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. \
If tool results for the question you dont know does not provide a related answer, say 'I don't know' and ask what you would need , for exmple if someone asks whats your favorite movie and tools does not provide any new information just say i dont know"

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [23]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = openai.chat.completions.create(model='gpt-5-nano', messages=messages, tools=tools, tool_choice="auto")

        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            print("tool called")
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content
    

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


tool called
Tool called: record_unknown_question with this arguments: {'question': "What is Soroush's favorite color?"}
Pushing : Recording this question: What is Soroush's favorite color? which i could not provide any answer
